In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import numpy as np
import pandas as pd
from keras.models import load_model
from matplotlib import pyplot as plt
import os
from Make_X3 import made_x
from keras.utils import np_utils
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


dir = 'ohlcv/'
ohlcv_list = os.listdir(dir)

if __name__ == '__main__':

    input_data_length = 54
    # model_num = input('Press model num : ')
    model_num = 20

    #           PARAMS           #
    check_span = 30
    Range_fluc = 1.035
    get_fig = 0

    #       Make folder      #
    try:
        os.mkdir('./pred_ohlcv/%s_%s/' % (input_data_length, model_num))
    except Exception as e:
        pass

    try:
        os.mkdir('./Figure_pred/%s_%s/' % (input_data_length, model_num))
    except Exception as e:
        pass

    except_list = os.listdir('./pred_ohlcv/%s_%s' % (input_data_length, model_num))

    #       LOAD MODEL      #
    model = load_model('./model/rapid_ascending %s_%s.hdf5' % (input_data_length, model_num))

    for file in ohlcv_list:

        #         if file in except_list:
        #             continue

        print('loading %s' % file)

        try:
            X_test, Y_test, sliced_ohlcv = made_x(file, input_data_length, model_num, check_span, Range_fluc, 0)

        except Exception as e:
            print('Error in getting data from made_x :', e)
            continue

        closeprice = np.roll(np.array(list(map(lambda x: x[-1][[1]][0], X_test))), -1)
        OBV = np.roll(np.array(list(map(lambda x: x[-1][[5]][0], X_test))), -1)

        # dataX 에 담겨있는 value 에 [-1] : 바로 이전의 행 x[-1][:].shape = (1, 6)
        # sliced_ohlcv = np.array(list(map(lambda x: x[-1][:], X_test)))
        # print(sliced_ohlcv)
        # quit()

        if len(X_test) != 0:

            X_test = np.array(X_test)

            row = X_test.shape[1]
            col = X_test.shape[2]

            X_test = X_test.astype('float32').reshape(-1, row, col, 1)
            # print(X_test.shape)

            Y_pred_ = model.predict(X_test, verbose=1) 
            
            max_value = np.max(Y_pred_, axis=0)
            limit_line = 0.9
            Y_pred = np.zeros(len(Y_pred_))
            for i in range(len(Y_pred_)):
                if Y_pred_[i][1] > max_value[1] * limit_line:
                    Y_pred[i] = 1
                elif Y_pred_[i][2] > max_value[2] * limit_line:
                    Y_pred[i] = 2
                elif Y_pred_[i][3] > max_value[3] * limit_line:
                    Y_pred[i] = 3
#             print(Y_pred)
#             quit()

            #       Save Pred_ohlcv      #
            #   기존에 pybithumb 을 통해서 제공되던 ohlcv 와는 조금 다르다 >> 이전 데이터와 현재 y 데이터 행이 같다.
            sliced_Y = Y_pred.reshape(-1, 1)
            pred_ohlcv = np.concatenate((sliced_ohlcv, sliced_Y), axis=1)  # axis=1 가로로 합친다

            #   col 이 7이 아닌 데이터 걸러주기
            try:
                pred_ohlcv_df = pd.DataFrame(pred_ohlcv,
                                             columns=['open', 'close', 'high', 'low', 'volume', 'OBV', 'trade_state'])

            except Exception as e:
                print(e)
                continue
            # print(pred_ohlcv_df.tail(20))
            # quit()
            pred_ohlcv_df.to_excel('./pred_ohlcv/%s_%s/%s' % (input_data_length, model_num, file))

            # Y_pred 에 1 이 존재하면, Plot Comparing
            if get_fig == 1:

                spanlist = []
                spanlist_low = []
                spanlist_high = []

                for m in range(len(Y_pred)):
                    if (Y_pred[m] > 0.5) and (Y_pred[m] < 1.5):
                        if m + 1 < len(Y_pred):
                            spanlist.append((m, m + 1))
                        else:
                            spanlist.append((m - 1, m))

                for m in range(len(Y_pred)):
                    if (Y_pred[m] > 1.5) and (Y_pred[m] < 2.5):
                        if m + 1 < len(Y_pred):
                            spanlist_low.append((m, m + 1))
                        else:
                            spanlist_low.append((m - 1, m))

                for m in range(len(Y_pred)):
                    if (Y_pred[m] > 2.5) and (Y_pred[m] < 3.5):
                        if m + 1 < len(Y_pred):
                            spanlist_high.append((m, m + 1))
                        else:
                            spanlist_high.append((m - 1, m))

                plt.subplot(311)
                plt.plot(closeprice, 'r', label='close')
                plt.plot(OBV, 'b', label='OBV')
                plt.legend(loc='upper right')
                for i in range(len(spanlist)):
                    plt.axvspan(spanlist[i][0], spanlist[i][1], facecolor='g', alpha=0.5)

                plt.subplot(312)
                # plt.subplot(313)
                plt.plot(closeprice, 'r', label='close')
                plt.plot(OBV, 'b', label='OBV')
                plt.legend(loc='upper right')
                for i in range(len(spanlist_low)):
                    plt.axvspan(spanlist_low[i][0], spanlist_low[i][1], facecolor='m', alpha=0.5)

                plt.subplot(313)
                # plt.subplot(313)
                plt.plot(closeprice, 'r', label='close')
                plt.plot(OBV, 'b', label='OBV')
                plt.legend(loc='upper right')
                for i in range(len(spanlist_high)):
                    plt.axvspan(spanlist_high[i][0], spanlist_high[i][1], facecolor='c', alpha=0.5)

                Date = file.split()[0]
                Coin = file.split()[1].split('.')[0]
                plt.savefig('./Figure_pred/%s_%s/%s %s.png' % (input_data_length, model_num, Date, Coin), dpi=500)
                plt.close()







loading 2020-01-16 XLM ohlcv.xlsx
1005/1005 [==============================] - 0s 163us/step
loading 2020-01-23 TRX ohlcv.xlsx
725/725 [==============================] - 0s 43us/step
loading 2020-01-16 SNT ohlcv.xlsx
217/217 [==============================] - 0s 47us/step
loading 2020-01-19 LBA ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-20 CHR ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-11 FX ohlcv.xlsx
318/318 [==============================] - 0s 45us/step
loading 2020-01-22 LUNA ohlcv.xlsx
1324/1324 [==============================] - 0s 42us/step
loading 2020-01-16 LINK ohlcv.xlsx
996/996 [==============================] - 0s 42us/step
loading 2019-10-25 FAB ohlcv.xlsx
292/292 [==============================] - 0s 49us/step
loading 2020-01-13 FCT ohlcv.xlsx
207/207 [==============================] - 0s 49us/step
loading 2020-01-20 IPX ohlcv.xlsx
1354/1354 [=====================

Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-31 BHP ohlcv.xlsx
730/730 [==============================] - 0s 43us/step
loading 2019-10-30 RNT ohlcv.xlsx
151/151 [==============================] - 0s 49us/step
loading 2019-10-26 LTC ohlcv.xlsx
899/899 [==============================] - 0s 43us/step
loading 2019-10-18 WET ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-14 ETH ohlcv.xlsx
1001/1001 [==============================] - 0s 42us/step
loading 2019-11-24 VET ohlcv.xlsx
360/360 [==============================] - 0s 46us/step
loading 2020-01-11 ABT ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-23 ETH ohlcv.xlsx
1069/1069 [==============================] - 0s 42us/step
loading 2020-01-16 CHR ohlcv.xlsx
101/101 [==============================] - 0s 59us/step
loading 2020-01-25 CON ohlcv.xlsx
799/799 [==============================] - 0s 42us/ste

Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-17 AE ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-15 QTUM ohlcv.xlsx
924/924 [==============================] - 0s 298us/step
loading 2020-01-22 WPX ohlcv.xlsx
1062/1062 [==============================] - 0s 231us/step
loading 2019-11-24 WET ohlcv.xlsx
450/450 [==============================] - 0s 294us/step
loading 2020-01-20 TRV ohlcv.xlsx
332/332 [==============================] - 0s 326us/step
loading 2020-01-15 BAT ohlcv.xlsx
167/167 [==============================] - 0s 318us/step
loading 2019-10-19 QKC ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-23 PAY ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-21 LOOM ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-16 GXC ohlcv.xlsx
702/702 [=============

Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-20 WAVES ohlcv.xlsx
147/147 [==============================] - 0s 51us/step
loading 2020-01-11 CHR ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-22 CON ohlcv.xlsx
1043/1043 [==============================] - 0s 42us/step
loading 2019-10-25 OMG ohlcv.xlsx
264/264 [==============================] - 0s 48us/step
loading 2020-01-11 TRUE ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-06 ITC ohlcv.xlsx
382/382 [==============================] - 0s 44us/step
loading 2019-10-16 XVG ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-25 MTL ohlcv.xlsx
273/273 [==============================] - 0s 47us/step
loading 2019-10-29 MXC ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-24 ABT ohlcv.xlsx
113/113 [================

189/189 [==============================] - 0s 45us/step
loading 2020-01-24 BTG ohlcv.xlsx
789/789 [==============================] - 0s 42us/step
loading 2020-01-20 ZRX ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-25 PIVX ohlcv.xlsx
159/159 [==============================] - 0s 48us/step
loading 2019-10-15 PPT ohlcv.xlsx
586/586 [==============================] - 0s 44us/step
loading 2020-01-23 BSV ohlcv.xlsx
1252/1252 [==============================] - 0s 41us/step
loading 2020-01-25 NPXS ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-17 XVG ohlcv.xlsx
151/151 [==============================] - 0s 50us/step
loading 2019-10-17 XEM ohlcv.xlsx
367/367 [==============================] - 0s 44us/step
loading 2019-10-30 TRV ohlcv.xlsx
497/497 [==============================] - 0s 44us/step
loading 2019-11-14 BAT ohlcv.xlsx
176/176 [==============================] - 0s 48us/step
loading 2020

...

121/121 [==============================] - 0s 56us/step
loading 2019-10-26 PIVX ohlcv.xlsx
531/531 [==============================] - 0s 47us/step
loading 2019-10-29 SNT ohlcv.xlsx
148/148 [==============================] - 0s 53us/step
loading 2019-10-29 CMT ohlcv.xlsx
299/299 [==============================] - 0s 48us/step
loading 2020-01-25 XVG ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-26 BAT ohlcv.xlsx
283/283 [==============================] - 0s 48us/step
loading 2020-01-11 BTC ohlcv.xlsx
1348/1348 [==============================] - 0s 45us/step
loading 2020-01-15 CHR ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-26 GXC ohlcv.xlsx
239/239 [==============================] - 0s 49us/step
loading 2019-10-19 DASH ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-12 STRAT ohlcv.xlsx
Error in getting data from made_x : 'NoneType' objec

134/134 [==============================] - 0s 57us/step
loading 2019-10-21 ZRX ohlcv.xlsx
605/605 [==============================] - 0s 45us/step
loading 2019-10-30 XVG ohlcv.xlsx
234/234 [==============================] - 0s 51us/step
loading 2019-10-28 XRP ohlcv.xlsx
1354/1354 [==============================] - 0s 44us/step
loading 2020-01-23 QKC ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-20 FZZ ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-11 VALOR ohlcv.xlsx
114/114 [==============================] - 0s 59us/step
loading 2020-01-16 NPXS ohlcv.xlsx
251/251 [==============================] - 0s 48us/step
loading 2019-10-27 MITH ohlcv.xlsx
133/133 [==============================] - 0s 60us/step
loading 2019-10-18 WTC ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-14 NPXS ohlcv.xlsx
Error in getting data from made_x : 'NoneType' obje

847/847 [==============================] - 0s 44us/step
loading 2020-01-25 AMO ohlcv.xlsx
618/618 [==============================] - 0s 46us/step
loading 2020-01-26 WPX ohlcv.xlsx
338/338 [==============================] - 0s 48us/step
loading 2019-10-30 CTXC ohlcv.xlsx
161/161 [==============================] - 0s 59us/step
loading 2019-10-15 BTG ohlcv.xlsx
172/172 [==============================] - 0s 54us/step
loading 2019-10-28 WTC ohlcv.xlsx
492/492 [==============================] - 0s 48us/step
loading 2019-11-01 STRAT ohlcv.xlsx
221/221 [==============================] - 0s 48us/step
loading 2020-01-11 MBL ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-19 THETA ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-05 CMT ohlcv.xlsx
251/251 [==============================] - 0s 49us/step
loading 2020-01-11 LBA ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterab

Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-11-01 OGO ohlcv.xlsx
190/190 [==============================] - 0s 52us/step
loading 2019-10-24 MIX ohlcv.xlsx
502/502 [==============================] - 0s 46us/step
loading 2020-01-14 CRO ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-21 FZZ ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-14 FCT ohlcv.xlsx
190/190 [==============================] - 0s 51us/step
loading 2020-01-11 IOST ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-15 PCM ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-11-17 FX ohlcv.xlsx
341/341 [==============================] - 0s 48us/step
loading 2019-11-14 MCO ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-26 THETA ohlcv.xlsx
Error in getti

277/277 [==============================] - 0s 49us/step
loading 2020-01-22 PPT ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-10-26 MTL ohlcv.xlsx
715/715 [==============================] - 0s 45us/step
loading 2020-01-21 KNC ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-21 PIVX ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2020-01-20 PPT ohlcv.xlsx
107/107 [==============================] - 0s 59us/step
loading 2019-10-21 LRC ohlcv.xlsx
Error in getting data from made_x : 'NoneType' object is not iterable
loading 2019-11-02 RNT ohlcv.xlsx
559/559 [==============================] - 0s 46us/step
loading 2020-01-21 TMTG ohlcv.xlsx
958/958 [==============================] - 0s 43us/step
loading 2020-01-22 APIS ohlcv.xlsx
161/161 [==============================] - 0s 57us/step
loading 2019-10-22 THETA ohlcv.xlsx
Error in getting data from made_x : 'No